In [8]:
import os
from tensorflow.keras.datasets import imdb
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
tf.random.set_seed(22)
np.random.seed(22)
assert tf.__version__.startswith('2.')

In [18]:
batch_size = 128
total_words = 10000
max_review_len = 80
embedding_len = 100

In [19]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=total_words)

In [20]:
print(x_train.shape)
print(x_test.shape)

(25000,)
(25000,)


In [21]:
pad_x_train = pad_sequences(x_train, maxlen=max_review_len)
pad_x_test = pad_sequences(x_test, maxlen=max_review_len)

In [25]:
train_data = tf.data.Dataset.from_tensor_slices((pad_x_train, y_train))
train_data = train_data.shuffle(10000).batch(batch_size, drop_remainder=True)
test_data = tf.data.Dataset.from_tensor_slices((pad_x_test, y_test))
test_data = test_data.batch(batch_size, drop_remainder=True)
print('x_train_shape:', x_train.shape, tf.reduce_max(y_train), tf.reduce_min(y_train))
print('x_test_shape:', x_test.shape)

x_train_shape: (25000,) tf.Tensor(1, shape=(), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
x_test_shape: (25000,)


In [38]:
sample = next(iter(test_data))
print(sample[0].shape)

(128, 80)


In [39]:
class LSTM_Build(tf.keras.Model):
    def __init__(self, units):
        super(LSTM_Build, self).__init__()
        
        self.state0 = [tf.zeros([batch_size, units]), tf.zeros([batch_size, units])]
        self.state1 = [tf.zeros([batch_size, units]), tf.zeros([batch_size, units])]
        
        self.embedding = tf.keras.layers.Embedding(total_words, embedding_len, input_length=max_review_len)
        self.RNNCell0 = tf.keras.layers.LSTMCell(units, dropout=0.5)
        self.RNNCell1 = tf.keras.layers.LSTMCell(units, dropout=0.5)
        self.outlayer = tf.keras.layers.Dense(1)
    
    def call(self, inputs, training=None):

        x = inputs
        x = self.embedding(x)
        state0 = self.state0
        state1 = self.state1
        for word in tf.unstack(x, axis=1):    

            out0, state0 = self.RNNCell0(word, state0, training)
            out1, state1 = self.RNNCell1(out0, state1, training)

        x = self.outlayer(out1)
        prob = tf.sigmoid(x)
        return prob

In [40]:
tf.random.set_seed(22)
np.random.seed(22)

In [41]:
import time
units = 64
epochs = 4
t0 = time.time()

model = LSTM_Build(units)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.losses.BinaryCrossentropy(),
              metrics=['accuracy'],
              experimental_run_tf_function=False)

In [42]:
model.fit(train_data, epochs=epochs, validation_data=test_data, validation_freq=2)

Epoch 1/4
195/195 [==============================] - 61s 178ms/step - loss: 0.4842 - accuracy: 0.7508
Epoch 2/4
195/195 [==============================] - 68s 347ms/step - loss: 0.3135 - accuracy: 0.8695 - val_loss: 0.3554 - val_accuracy: 0.8428
Epoch 3/4
195/195 [==============================] - 60s 309ms/step - loss: 0.2608 - accuracy: 0.8959
Epoch 4/4
195/195 [==============================] - 131s 673ms/step - loss: 0.2203 - accuracy: 0.9149 - val_loss: 0.4183 - val_accuracy: 0.8294


In [43]:
print("훈련 데이터셋 평가...")
(loss, accuracy) = model.evaluate(train_data, verbose=1)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
print("테스트 데이터셋 평가...")
(loss, accuracy) = model.evaluate(test_data, verbose=1)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

t1 = time.time()
print('시간:', t1-t0)

훈련 데이터셋 평가...
195/195 [==============================] - 36s 184ms/step - loss: 0.1424 - accuracy: 0.9548
loss=0.1424, accuracy: 95.4768%
테스트 데이터셋 평가...
195/195 [==============================] - 33s 170ms/step - loss: 0.4183 - accuracy: 0.8294
loss=0.4183, accuracy: 82.9367%
시간: 460.7606542110443


In [45]:
class LSTM_Build(tf.keras.Model):
    def __init__(self, units):
        super(LSTM_Build, self).__init__()
        
        self.state0 = [tf.zeros([batch_size, units]), tf.zeros([batch_size, units])]
        self.state1 = [tf.zeros([batch_size, units]), tf.zeros([batch_size, units])]
        
        self.embedding = tf.keras.layers.Embedding(total_words, embedding_len, input_length=max_review_len)
        self.rnn = tf.keras.Sequential([
            tf.keras.layers.LSTM(units, dropout=0.5, return_sequences=True, unroll=True),
            tf.keras.layers.LSTM(units, dropout=0.5, unroll=True)  ])
        self.outlayer = tf.keras.layers.Dense(1)
    
    def call(self, inputs, training=None):
        x = inputs
        x = self.embedding(x)
        x = self.rnn(x)

        x = self.outlayer(x)
        prob = tf.sigmoid(x)
        return prob

In [47]:
import time
units = 64
epochs = 4
t0 = time.time()

model = LSTM_Build(units)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.losses.BinaryCrossentropy(),
              metrics=['accuracy'],
              experimental_run_tf_function=False)

In [48]:
model.fit(train_data, epochs=epochs, validation_data=test_data, validation_freq=2)

Epoch 1/4
195/195 [==============================] - 51s 155ms/step - loss: 0.4836 - accuracy: 0.7517
Epoch 2/4
195/195 [==============================] - 44s 228ms/step - loss: 0.3127 - accuracy: 0.8689 - val_loss: 0.3617 - val_accuracy: 0.8412
Epoch 3/4
195/195 [==============================] - 30s 155ms/step - loss: 0.2596 - accuracy: 0.8969
Epoch 4/4
195/195 [==============================] - 40s 205ms/step - loss: 0.2173 - accuracy: 0.9145 - val_loss: 0.4473 - val_accuracy: 0.8294


In [49]:
print("훈련 데이터셋 평가...")
(loss, accuracy) = model.evaluate(train_data, verbose=1)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
print("테스트 데이터셋 평가...")
(loss, accuracy) = model.evaluate(test_data, verbose=1)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

t1 = time.time()
print('시간:', t1-t0)

훈련 데이터셋 평가...
195/195 [==============================] - 12s 61ms/step - loss: 0.1408 - accuracy: 0.9535
loss=0.1408, accuracy: 95.3526%
테스트 데이터셋 평가...
195/195 [==============================] - 13s 64ms/step - loss: 0.4473 - accuracy: 0.8294
loss=0.4473, accuracy: 82.9407%
시간: 485.81644558906555


In [50]:
model.summary()

Model: "lstm__build_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      multiple                  1000000   
_________________________________________________________________
sequential (Sequential)      (128, 64)                 75264     
_________________________________________________________________
dense_8 (Dense)              multiple                  65        
Total params: 1,075,329
Trainable params: 1,075,329
Non-trainable params: 0
_________________________________________________________________


In [51]:
class GRU_Build(tf.keras.Model):
    def __init__(self, units):
        super(GRU_Build, self).__init__()
        
        self.state0 = [tf.zeros([batch_size, units]), tf.zeros([batch_size, units])]
        self.state1 = [tf.zeros([batch_size, units]), tf.zeros([batch_size, units])]
        
        self.embedding = tf.keras.layers.Embedding(total_words, embedding_len, input_length=max_review_len)
        self.RNNCell0 = tf.keras.layers.GRUCell(units, dropout=0.5)
        self.RNNCell1 = tf.keras.layers.GRUCell(units, dropout=0.5)
        self.outlayer = tf.keras.layers.Dense(1)
    
    def call(self, inputs, training=None):

        x = inputs
        x = self.embedding(x)
        state0 = self.state0
        state1 = self.state1
        for word in tf.unstack(x, axis=1):    

            out0, state0 = self.RNNCell0(word, state0, training)
            out1, state1 = self.RNNCell1(out0, state1, training)

        x = self.outlayer(out1)
        prob = tf.sigmoid(x)
        return prob
    


In [52]:
import time
units = 64
epochs = 4
t0 = time.time()

model = GRU_Build(units)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.losses.BinaryCrossentropy(),
              metrics=['accuracy'],
              experimental_run_tf_function=False)

In [53]:
model.fit(train_data, epochs=epochs, validation_data=test_data, validation_freq=2)

Epoch 1/4
195/195 [==============================] - 63s 185ms/step - loss: 0.5146 - accuracy: 0.7200
Epoch 2/4
195/195 [==============================] - 57s 292ms/step - loss: 0.3182 - accuracy: 0.8671 - val_loss: 0.3568 - val_accuracy: 0.8423
Epoch 3/4
195/195 [==============================] - 37s 188ms/step - loss: 0.2608 - accuracy: 0.8957s - loss: 0.2604 - 
Epoch 4/4
195/195 [==============================] - 49s 250ms/step - loss: 0.2200 - accuracy: 0.9150 - val_loss: 0.3826 - val_accuracy: 0.8320


In [54]:
print("GRU 훈련 데이터셋 평가...")
(loss, accuracy) = model.evaluate(train_data, verbose=1)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
print("GRU 테스트 데이터셋 평가...")
(loss, accuracy) = model.evaluate(test_data, verbose=1)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

t1 = time.time()
print('시간:', t1-t0)

GRU 훈련 데이터셋 평가...
195/195 [==============================] - 15s 76ms/step - loss: 0.1740 - accuracy: 0.9500
loss=0.1740, accuracy: 94.9960%
GRU 테스트 데이터셋 평가...
195/195 [==============================] - 15s 76ms/step - loss: 0.3826 - accuracy: 0.8320
loss=0.3826, accuracy: 83.1971%
시간: 239.37457656860352


In [55]:
class GRU_Build(tf.keras.Model):
    def __init__(self, units):
        super(GRU_Build, self).__init__()
        
        self.state0 = [tf.zeros([batch_size, units]), tf.zeros([batch_size, units])]
        self.state1 = [tf.zeros([batch_size, units]), tf.zeros([batch_size, units])]
        
        self.embedding = tf.keras.layers.Embedding(total_words, embedding_len, input_length=max_review_len)
        self.rnn = tf.keras.Sequential([
            tf.keras.layers.GRU(units, dropout=0.5, return_sequences=True, unroll=True),
            tf.keras.layers.GRU(units, dropout=0.5, unroll=True)  ])
        self.outlayer = tf.keras.layers.Dense(1)
    
    def call(self, inputs, training=None):
        x = inputs
        x = self.embedding(x)
        x = self.rnn(x)

        x = self.outlayer(x)
        prob = tf.sigmoid(x)
        return prob

    


In [56]:
import time
units = 64
epochs = 4
t0 = time.time()

model = GRU_Build(units)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.losses.BinaryCrossentropy(),
              metrics=['accuracy'],
              experimental_run_tf_function=False)

In [57]:
model.fit(train_data, epochs=epochs, validation_data=test_data, validation_freq=2)

Epoch 1/4
195/195 [==============================] - 63s 183ms/step - loss: 0.5190 - accuracy: 0.7273
Epoch 2/4
195/195 [==============================] - 53s 274ms/step - loss: 0.3280 - accuracy: 0.8624 - val_loss: 0.3710 - val_accuracy: 0.8385
Epoch 3/4
195/195 [==============================] - 35s 180ms/step - loss: 0.2692 - accuracy: 0.8916
Epoch 4/4
195/195 [==============================] - 48s 245ms/step - loss: 0.2316 - accuracy: 0.9100 - val_loss: 0.3970 - val_accuracy: 0.8332


In [58]:
print("GRU 훈련 데이터셋 평가...")
(loss, accuracy) = model.evaluate(train_data, verbose=1)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
print("GRU 테스트 데이터셋 평가...")
(loss, accuracy) = model.evaluate(test_data, verbose=1)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

t1 = time.time()
print('시간:', t1-t0)

GRU 훈련 데이터셋 평가...
195/195 [==============================] - 15s 77ms/step - loss: 0.1699 - accuracy: 0.9421
loss=0.1699, accuracy: 94.2107%
GRU 테스트 데이터셋 평가...
195/195 [==============================] - 16s 81ms/step - loss: 0.3970 - accuracy: 0.8332
loss=0.3970, accuracy: 83.3213%
시간: 299.01604771614075


In [2]:

# 양방향 RNn

In [3]:
import os
from tensorflow.keras.datasets import imdb
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense , Dropout , Embedding, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
word_num = 10000
maxien = 200
batch_size = 128

In [8]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=word_num)

In [10]:
x_train = pad_sequences(x_train, maxlen=maxien)
x_test = pad_sequences(x_test, maxlen=maxien)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [12]:
model = Sequential()
model.add(Embedding(word_num,128,input_lengh=maxien))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

TypeError: ('Keyword argument not understood:', 'input_lengh')

In [ ]:
model.compile(loss = 'binary_crossentropy',optimizer = 'adam', metrics['accuracy'])